# Test Stability of U 
#### (Using the Tonsil dataset)

## Import Libraries

First, we import the neccessary libraries.

In [1]:
# Import libraries
import anndata as ad
from pathlib import Path
import errno
import os
import numpy as np
import pandas as pd
# import seaborn as sns

## Helper Functions

Next, we import some helper functions including the cisi segementation training function.

In [2]:
# Helper fncs
import helpers.analysis_utils


## CISI
# Import system libraries to configure code directory as module
from os.path import dirname, abspath, join
import sys

# Find code directory relative to our directory
THIS_DIR = dirname('__file__')
CODE_DIR = abspath(join(THIS_DIR, '..', 'code'))
# Add code directory to systems paths
sys.path.append(CODE_DIR)

# Import dictionary training fnc. (smaf)
from compute_dictionary import smaf


# Define fnc to compare correlation matrices
def compare_cor(A, B):
    return np.sum(abs(A - B))

## Inputs

In the first part we specify the paths to the input files (.h5ad files created from R) and where the outputs should be stored.

In [4]:
# Specify input paths
training_data_path = Path('/mnt/bb_dqbm_volume')
# spe_path = Path(os.path.join(training_data_path, 
#                             'data/Tonsil_th152/preprocessed_data/spe.h5ad'))
spe_path = Path(os.path.join(training_data_path, 
                             'data/Immucan_lung/Lung_sce.h5ad'))
# data_name = 'Tonsil_th152'
data_name = 'Immucan_lung'


# Specify output path
out_path = Path(os.path.join(training_data_path, 
                             'analysis', data_name, 'tests/test_U_stability'))
out_path_k = Path(os.path.join(training_data_path, 
                               'analysis', data_name, 'tests/test_U_stability_par/k'))
out_path_maxItr = Path(os.path.join(training_data_path, 
                               'analysis', data_name, 'tests/test_U_stability_par/maxItr'))
out_path_norm = Path(os.path.join(training_data_path, 
                               'analysis', data_name, 'tests/test_U_stability_par/transformation'))
out_path_dictSize = Path(os.path.join(training_data_path, 
                               'analysis', data_name, 'tests/test_U_stability_par/dictSize'))

# Create output directory if it doesn't exist
out_path.mkdir(parents=True, exist_ok=True)
out_path_k.mkdir(parents=True, exist_ok=True)
out_path_maxItr.mkdir(parents=True, exist_ok=True)
out_path_norm.mkdir(parents=True, exist_ok=True)
out_path_dictSize.mkdir(parents=True, exist_ok=True)

In [5]:
# Check that input files/dictionary exist
if not helpers.analysis_utils.is_valid_file(spe_path, ['.h5ad']):
    # If file is not found, throw error
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT),
                            spe_path)

Next, we read in the input files for training U and testing its stability. For this we have the Tonsil th152 dataset consisting of 5 ROIs and we read the data in once it has been processed by steinbock into segmented single cells.

In [6]:
# Read in SpatialExperiment converted to anndata by cellconverter in R
spe = ad.read_h5ad(spe_path)
spe = spe[:, ~spe.var.index.str.contains('Histone|Ir[0-9]|E-Cad', regex=True, case=False)]
print(spe)

View of AnnData object with n_obs × n_vars = 2322277 × 36
    obs: 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'area', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px', 'height_px', 'acquisition_id', 'image', 'Slide_id', 'slide_position', 'sample_name', 'ROI', 'sample_type', 'subbatch', 'cell_id', 'cell_labels', 'celltype', 'celltype_minor', 'TLS', 'distance_to_TLS', 'Tumor_mask', 'distance_to_Tumor', 'CN_30_15', 'CN_minor_30_15', 'clusters'
    var: 'channel', 'name', 'keep', 'ilastik', 'deepcell', 'Tube.Number', 'Target', 'Antibody.Clone', 'Stock.Concentration', 'Final.Concentration...Dilution', 'uL.to.add', 'tumorMask', 'channel_name'
    uns: 'X_name'
    layers: 'exprs', 'normalized', 'scaled'


Next, we specify the parameters we want to use to compute the dictionary and how many times to assess it.


In [8]:
# Set non-default parameters of smaf
d = 80
lda1 = 3
lda2 = 0.2

# Set parameters for stability analysis of U
n_sizes = 10
n_rep = 10
min_train_size = 1000

## Check different Training Sizes

In the following paragraphs, we compute the "ground truth" correlation between proteins in U by computing the correlation matrix of U of the full dataset n_rep times and taking its average. Then we compare this to the results of input data sets of different sizes (by taking the sum of the absolut differences between ground truth and each result).

### k = 3 (paper default)

In [ ]:
# Calculate "ground truth" by computing the average correlation matrix of the full dataset
cor_list = [-10.0] * n_rep 
for i in range(n_rep):
        U, W = smaf(spe, d, lda1, lda2, normalization='paper_norm', layer=None,
                    outpath=os.path.join(out_path, "k_3", str(spe.shape[0]), str(i)))
        cor_w_diag = np.corrcoef(U)
        # Remove diagonal
        cor_list[i] = cor_w_diag[~np.eye(len(cor_w_diag), dtype=bool)].reshape(len(cor_w_diag), -1)

# Calculate "ground truth" as average of the five runs of the full data set
cor_ground_truth = sum(cor_list) / n_rep


# Initialize list of lists to hold results of distances between correlation matrices
# cor_list_subset = [-10] * 6
# cor_list_subset[5] = [compare_cor(c, cor_ground_truth) for c in pages]
cor_list_subset = np.full((n_rep, n_sizes), 10.0)
cor_list_subset[:, (n_sizes-1)] = np.array([compare_cor(c, cor_ground_truth) for c in cor_list])

In [ ]:
## Calculate correlation matrix of the subsets of the dataset
# For different subset sizes of spe, caculate n_rep times U and compare how close its
# columnwise correlations are to the mean correlation matrix of the full dataset (excluding)
# the diagonal
j = 0
for s in np.linspace(min_train_size, spe.shape[0], n_sizes-1, dtype=int, endpoint=False):
    for i in range(n_rep):
        X_subset = spe[np.random.randint(spe.shape[0], size=(s)), ]
        
        U, W = smaf(X_subset, d, lda1, lda2, outpath=os.path.join(out_path, "k_3", 
                                                                  str(s), str(i)), 
                    normalization='paper_norm', layer=None)
        cor_w_diag = np.corrcoef(U)
        cor_wt_diag = cor_w_diag[~np.eye(len(cor_w_diag), dtype=bool)].reshape(len(cor_w_diag), -1)
        
        cor_list_subset[i, j] = compare_cor(cor_wt_diag, cor_ground_truth)
    
    j+=1
        

### k = 1 

In [ ]:
# Set k to 1
lda1 = 1

# Calculate "ground truth" by computing the average correlation matrix of the full dataset
cor_list_k1 = [-10.0] * n_rep 
for i in range(n_rep):
        U, W = smaf(spe, d, lda1, lda2, normalization='paper_norm', layer=None,
                    outpath=os.path.join(out_path, "k_1", str(spe.shape[0]), str(i)))
        cor_w_diag = np.corrcoef(U)
        # Remove diagonal
        cor_list_k1[i] = cor_w_diag[~np.eye(len(cor_w_diag), dtype=bool)].reshape(len(cor_w_diag), -1)

# Calculate "ground truth" as average of the five runs of the full data set
cor_ground_truth_k1 = sum(cor_list_k1) / n_rep


# Initialize list of lists to hold results of distances between correlation matrices
# cor_list_subset = [-10] * 6
# cor_list_subset[5] = [compare_cor(c, cor_ground_truth) for c in pages]
cor_list_subset_k1 = np.full((n_rep, n_sizes), 10.0)
cor_list_subset_k1[:, (n_sizes-1)] = np.array([compare_cor(c, cor_ground_truth_k1) for c in cor_list_k1])

In [ ]:
## Calculate correlation matrix of the subsets of the dataset
# For different subset sizes of spe, caculate n_rep times U and compare how close its
# columnwise correlations are to the mean correlation matrix of the full dataset (excluding)
# the diagonal
j = 0
for s in np.linspace(min_train_size, spe.shape[0], n_sizes-1, dtype=int, endpoint=False):
    for i in range(n_rep):
        X_subset = spe[np.random.randint(spe.shape[0], size=(s)), ]
        
        U, W = smaf(X_subset, d, lda1, lda2, outpath=os.path.join(out_path, "k_1", str(s), 
                                                                  str(i)), 
                    normalization='paper_norm', layer=None)
        cor_w_diag = np.corrcoef(U)
        cor_wt_diag = cor_w_diag[~np.eye(len(cor_w_diag), dtype=bool)].reshape(len(cor_w_diag), -1)
        
        cor_list_subset_k1[i, j] = compare_cor(cor_wt_diag, cor_ground_truth_k1)
    
    j+=1
        

In [ ]:
# Convert into pandas dataframe for easier handling
cor_pd = pd.DataFrame(cor_list_subset, 
                         columns=np.append(np.linspace(min_train_size, spe.shape[0], 
                                                       n_sizes-1, dtype=int, endpoint=False), 
                                           spe.shape[0]))
cor_pd_k1 = pd.DataFrame(cor_list_subset_k1, 
                         columns=np.append(np.linspace(min_train_size, spe.shape[0], 
                                                       n_sizes-1, dtype=int, endpoint=False), 
                                           spe.shape[0]))
# Melt dataframe into long format for plotting
# cor_pd_long = pd.melt(cor_pd, var_name='subset_size', value_name='distance')

# Use seaborns regression fnc of order 2 to plot results
# ax = sns.regplot(x="subset_size", y="distance", data=cor_pd_long, order=1)

## Check different Parameters
### k-sparsity (lda1)

Next, we have a look at how the dictionary U changes, for different values (1-10) of k (columnwise-sparsity constraint of W used when computing U in smaf()). 

In [ ]:
# Specify list of k to try (1 to 10)
k = np.arange(1, 10+1)
for i in k:
    for j in range(n_rep):
        U, W = smaf(spe, d, i, lda2, normalization='paper_norm', layer=None,
                    outpath=os.path.join(out_path_k, str(i), str(j)))
        

### maxItr (number of iterations to compute U)

Next, we have a look at how the dictionary U changes, if we change the number of iterations to compute U. 

In [ ]:
# Set k back to 3
lda1 = 3

# Specify list of itr to try (10 to 50)
itr = np.arange(10, 50+1, 10)
for i in itr:
    for j in range(n_rep):
        U, W = smaf(spe, d, lda1, lda2, maxItr=i, normalization='paper_norm', layer=None,
                    outpath=os.path.join(out_path_maxItr, str(i), str(j)))

### normalization (X normalization type)

Next, we have a look at how the dictionary U changes, if we change the type of normalization of X before training U. 

In [9]:
# Set k back to 1
lda1 = 1

# Specify list of itr to try (10 to 50)
itr = ['paper_norm', 'min_max_norm', 'none']
for i in itr:
    for j in range(n_rep):
        U, W = smaf(spe, d, lda1, lda2, maxItr=10, normalization=i, layer=None,
                    outpath=os.path.join(out_path_norm, str(i), str(j)))
        
        
itr = ['exprs', 'log_exprs'] #, 'normalized', 'scaled'] 
for i in itr:
    for j in range(n_rep):
        U, W = smaf(spe, d, lda1, lda2, maxItr=10, normalization='none', layer=i,
                    outpath=os.path.join(out_path_norm, str(i), str(j)))

KeyError: 'log_exprs'

Next, we have a look at how the dictionary U changes, if we change the type of normalization of X before training U. 

### dictionary size 

Next, we have a look at how the dictionary U changes, if we change the upper number of modules (dictionary size) to train U for. 

In [ ]:
# Specify list of # modules in dictionary from 10 to 100
dict_sizes = np.linspace(10, 100, 10, dtype=int, endpoint=True).tolist()
for i in dict_sizes:
    for j in range(n_rep):
        U, W = smaf(spe, d=i, lda1=lda1, lda2=lda2, maxItr=10, normalization='paper_norm', layer=None,
                    outpath=os.path.join(out_path_dictSize, str(i), str(j)))

## Save results

In [ ]:
cor_pd.to_csv(os.path.join(out_path, "k_3", "results.csv"))
cor_pd_k1.to_csv(os.path.join(out_path, "k_1", "results.csv"))